In [2]:
# coding:utf-8

import sys
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import jieba
from jieba import analyse
import numpy as np
from sklearn import metrics
from sklearn.svm import SVC
import random


def get_segment(all=False):
    csvfile = open('movie.csv', newline='')
    rows = csv.DictReader(csvfile)
    temps=[]
    for row in rows:
        seg_list = jieba.cut(row['名稱'] + " " + row['分類'] + " " + row['劇情介紹'],cut_all=False)
        line = ""
        for stri in seg_list:
            line = line + " " + stri
        line = line.replace('\'', '')
        temp={}
        temp['名稱'] = row['名稱']
        temp['分類'] = row['分類']
        temp['內容'] = line
        temps.append(temp)
    labels = ['名稱', '分類', '內容']
    with open('movie1.csv', 'w', encoding='utf-8-sig') as f:
      writer = csv.DictWriter(f, fieldnames=labels)
      writer.writeheader()
      for i in temps:
        writer.writerow(i)

def product_category():
  category={}
  category[1] = '動作'
  category[2] = '冒險'
  category[3] = '科幻'
  category[4] = '奇幻'
  category[5] = '劇情'
  category[6] = '犯罪'
  category[7] = '恐怖'
  category[8] = '懸疑/驚悚'
  category[9] = '喜劇'
  category[10] = '愛情'
  category[11] = '溫馨/家庭'
  category[12] = '動畫'
  category[13] = '戰爭'
  category[14] = '音樂/歌舞'
  category[15] = '歷史/傳記'
  category[16] = '紀錄片'
  category[17] = '勵志'
  category[18] = '武俠'
  category[19] = '影展'
  category[20] = '戲劇'
  category[21] = '影集'
  return category

get_segment()
category_ids = []
category = product_category()
test_typs = []
test_titles = []
rnds = []
test = []
csvfile = open('movie1.csv', newline='', encoding='utf-8-sig')
rows = csv.DictReader(csvfile)
line = ""
s = list(rows)

def product_test(s,test_titles,test_typs,test,rnds):
  k = 0
  while k < 500:
      rnd = random.randint(0,len(s)-1)
      if rnd not in rnds:
        title = dict(s[rnd])['名稱']
        test_titles.append(title)
        typ = dict(s[rnd])['分類']
        test_typs.append(typ)
        segment = dict(s[rnd])['內容']
        test.append(segment)
        rnds.append(rnd)
        k = k + 1
  return s,test_titles,test_typs,test,rnds

s,test_titles,test_typs,test,rnds = product_test(s,test_titles,test_typs,test,rnds)
train_typs = []
train_titles = []
train = []
line = ""
for i in range(0,len(s)-1):
  if i not in rnds:
      title = dict(s[i])['名稱']
      train_titles.append(title)
      typ = dict(s[i])['分類']
      train_typs.append(typ)
      segment = dict(s[i])['內容']
      train.append(segment)
      for j in range(1,22):
                if dict(s[i])['分類']==category[j]:
                  category_ids.append(j)


def classification(train, test, category_ids):
  vectorizer=CountVectorizer()
  train_mat = vectorizer.fit_transform(train)
  test_mat = vectorizer.transform(test)
  transformer=TfidfTransformer()
  train_tfidf=transformer.fit_transform(train_mat)
  test_tfidf=transformer.transform(test_mat)
  y = np.array(category_ids)
  model = SVC()
  model.fit(train_tfidf, y)
  predicted = model.predict(test_tfidf)
  return predicted

predicted = classification(train, test, category_ids)

def print_out(predicted, category, test_titles, test_typs):
  correct = 0
  for index in range(0, len(test_titles)):
        predicted_category = category[predicted[index]]
        print("predict title: %s <==============> predicted category: %s  true category: %s" % (test_titles[index], predicted_category, test_typs[index]))
        if predicted_category==test_typs[index]:
          correct += 1
  print("本次訓練的正確性為：%s" % (correct/500))

print_out(predicted, category, test_titles, test_typs)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.888 seconds.
Prefix dict has been built successfully.


predict title: 乘風破浪 <==============> predicted category: 劇情  true category: 劇情
predict title: 葡萄酒世界 <==============> predicted category: 劇情  true category: 紀錄片
predict title: K SEVEN STORIES Episode6「Circle Vision ～Nameless Song～」 <==============> predicted category: 劇情  true category: 動畫
predict title: 實尾島風雲 <==============> predicted category: 動作  true category: 動作
predict title: 盂蘭神功 <==============> predicted category: 恐怖  true category: 恐怖
predict title: 女狼嗨到趴 <==============> predicted category: 劇情  true category: 喜劇
predict title: 心的靜寂 <==============> predicted category: 劇情  true category: 劇情
predict title: 素人特工 <==============> predicted category: 動作  true category: 動作
predict title: 愛情初邂逅 <==============> predicted category: 劇情  true category: 愛情
predict title: 狂愛 聖彼得堡 <==============> predicted category: 劇情  true category: 愛情
predict title: 自殺突擊隊 <==============> predicted category: 動作  true category: 動作
predict title: 天之茶助 <==============> predicted category: 劇情  true categ